# （実装案）実験パイプライン作成支援

本セクションでは、実験の再現をするにあたって、コードの実行順（および入出力データ）を保持し、その順で実行する「実験パイプライン」の作成を支援します。

### ◆◆◆開発メモ◆◆◆

- 現在は最低限pipeline.yamlに必要な`tasks`以下のパラメータのみを書き出す実装。  
入出力データの定義は、キー名だけ書き出してユーザに埋めてもらうような方式が良いかも？
- また、実行されるコード側に定義しなければいけない実装はMarkdownによる手順だけの用意としている。（わかりにくいかも・・）

<!-- APIを利用したparameterセルの追加処理
 →書こうと思えば書けるが、ここまでする必要はある？
 refs: https://ohke.hateblo.jp/entry/2019/05/25/180000 -->

## 1. 作成準備

パイプラインの生成のため、まずはシステムにそれぞれのコードの関係を知らせるパラメータを定義します。  
以下の手順に沿って、それぞれのコードの関係を定義してください。

1. 該当するコードを開きます。
2. 上部ツールバーから「View→Cell Toolbar→Tags」を選択します。  
これにより、各セルの上部にタグを追加するフォームが表示されます。
3. １番目の実行箇所になる最上部にセルを１つ挿入します。
4. 3.で挿入したセルのタグ定義フォームに、`parameters`と入力し、`Add tag`ボタンを押下します。  
これにより、`parameters`タグが挿入されます。
5. 3.で挿入したセルに、以下の通りコードを記述します。そのファイルよりも前に実行されるべきコードがある場合、そのファイル名を配列で`upstream = ['{前に実行するファイル名}']`のように定義してください。
```py
upstream = None
product = None
```

## 2. パイプライン設定ファイルの生成

このステップでは、パイプライン全体の実行に必要な設定ファイルをシステムが作成します。以下に挿入されている`from ruamel.yaml import YAML`で始まるセルを実行してください。コードファイルを読み取り、設定ファイル`pipeline.yaml`を生成します。

In [ ]:
from ruamel.yaml import YAML
import os
import glob

files = glob.glob(os.getcwd()+'/*')
code_dict = {}

# YAMLのベースとなる辞書
yml = {'tasks': []}

for file in files:
    # ソースリストに載っている拡張子なら辞書作成する処理に修正したい
    if os.path.splitext(file)[1] == '.ipynb':
        code_dict['source']  = os.path.basename(file)
        code_dict['product'] = 'output/log_' + os.path.basename(file)
        yml['tasks'].append(code_dict)
        code_dict = {}

with open('pipeline.yaml', 'w') as file:
    yaml = YAML()
    yaml.default_flow_style = False
    yaml.dump(yml, file)

## 3. データガバナンス機能へ書き戻し

以上の変更をデータガバナンス機能の研究リポジトリにも反映させます。以下のセルを実行してください。

In [ ]:
# datalad save & push と同等の処理を行うセル